In [2]:
import joblib
import pandas as pd
import scipy
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
vectorizer = joblib.load("model/vectorizer.pkl")
scaler = joblib.load("model/scaler.pkl")
combined_recom_features = joblib.load("model/combined_features.pkl")
gunung = joblib.load("model/gunung_data.pkl")

In [49]:
def rekomendasikan_gunung(input_lokasi, input_ketinggian, top_n=5, similarity_threshold=0.3):
    # Validasi
    if not isinstance(input_lokasi, str):
        raise ValueError("Input lokasi harus berupa teks.")
    if input_lokasi.isnumeric():
        raise ValueError("Input lokasi tidak boleh hanya berisi angka.")
    if not isinstance(input_ketinggian, (int, float)):
        raise ValueError("Input ketinggian harus berupa angka.")

    # Transform input
    input_lokasi_vec = vectorizer.transform([input_lokasi])
    input_ketinggian_scaled = scaler.transform(pd.DataFrame([[input_ketinggian]], columns=["Ketinggian (dpl)"]))
    input_combined = scipy.sparse.hstack([input_lokasi_vec, input_ketinggian_scaled])

    # Cosine similarity
    similarity_scores = cosine_similarity(input_combined, combined_recom_features).flatten()

    # Filter gunung yang aksesnya buka
    filter_akses = gunung['Akses'] == 'Buka'
    qualified_indices = [
        i for i, score in enumerate(similarity_scores)
        if score >= similarity_threshold and filter_akses[i]
    ]

    final_indices = sorted(qualified_indices, key=lambda i: similarity_scores[i], reverse=True)[:top_n]
    hasil = gunung.iloc[final_indices][['Nama', 'Provinsi', 'Ketinggian (dpl)', 'Akses']]
    
    return hasil.reset_index(drop=True)

In [50]:
def get_input_dict():
    lokasi = input("Masukkan lokasi: ")
    ketinggian = float(input("Masukkan ketinggian ideal (dalam mdpl): "))
    return {
        "Lokasi": lokasi,
        "Ketinggian (dpl)": ketinggian
    }

new_input = pd.DataFrame([get_input_dict()])

print("\nData input:")
for key, value in new_input.iloc[0].items():
    print(f"{key}: {value}")

lokasi_user = new_input["Lokasi"].values[0]
ketinggian_user = new_input["Ketinggian (dpl)"].values[0]


Data input:
Lokasi: jambi
Ketinggian (dpl): 2000.0


In [51]:
rekomendasi = rekomendasikan_gunung(lokasi_user, ketinggian_user)

if rekomendasi.empty:
    print("\nTidak ada rekomendasi yang ditemukan.")
else:
    print("\nRekomendasi gunung:")
    print(rekomendasi)



Rekomendasi gunung:
                     Nama Provinsi  Ketinggian (dpl) Akses
0  Gunung Sumbing (Jambi)    Jambi              2507  Buka
1          Gunung Masurai    Jambi              2980  Buka
2          Gunung Kerinci    Jambi              3805  Buka
3        Gunung Hulu Nilo    Jambi              2424  Buka
4           Gunung Kunyit    Jambi              2151  Buka
